In [31]:
import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder
from IPython.display import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
# import tensorflow as tf
# from sklearn.preprocessing import OneHotEncoder

In [8]:
!ls

Fireball_And_Bolide_Reports.csv  Near-Earth_Comets_-_Orbital_Elements.csv
GLC03122015.csv			 Untitled.ipynb
Meteorite_Landings.csv


In [166]:
# db = pd.read_csv('Fireball_And_Bolide_Reports.csv')
# db = pd.read_csv('GLC03122015.csv')
db = pd.read_csv('Near-Earth_Comets_-_Orbital_Elements.csv')
# db = pd.read_csv('Meteorite_Landings.csv')
# pd.set_option('display.max_rows', db.shape[0]+1)

In [167]:
# db['nan number'] = db.isnull().sum(1)
# db = db.sort_values(by=['nan number'])
# db.dtypes

In [168]:
labelencoder = LabelEncoder()
for label in db.select_dtypes(include='object').columns.tolist():
    st = time.time()
    mask = ~db[label].isnull()
    db[label][mask] = labelencoder.fit_transform(db[label][mask])
    ed = time.time()
#     print(ed-st)
    if len(db[label][mask]) * 0.1 < db[label][mask].max():
        db = db.drop(columns=[label])

/home/kevinlin/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [169]:
class Net(torch.nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        add_block = []
        add_block += [nn.Linear(num_classes, num_classes*10)]
        add_block += [nn.ReLU()]
        for i in range(1):
            add_block += [nn.Linear(num_classes*10, num_classes*10)]
#             add_block += [nn.BatchNorm1d(num_classes*10)]
            add_block += [nn.ReLU()]
            
        self.add_block = nn.Sequential(*add_block)
        self.fc = nn.Linear(num_classes*10, num_classes)
        
    def forward(self, x):
        x = self.add_block(x)
        x = self.fc(x)
        return x
    
def train(model, train_matrix, masked_num, optimizer, epochs):
    
    
    model.train()
    r, c = train_matrix.shape[0], train_matrix.shape[1]
    gt = torch.from_numpy(train_matrix.copy().astype(np.float32)).cuda()
    for i in range(epochs):
        data = train_matrix.copy()
        for j in range(r):
            indices = np.random.choice(c, replace=False, size=masked_num)
            data[j,indices] = 0
        data = torch.from_numpy(data.astype(np.float32)).cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.l1_loss(output, gt)
        loss.backward()
        optimizer.step()
        
        if i%1000 == 0:
            print(loss.item())
#             print(data)
#         if batch_idx % args.log_interval == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))

def test(model, test_matrix):
    data = torch.from_numpy(test_matrix.astype(np.float32)).cuda()
    with torch.no_grad():
        output = model(data)
    return output.detach().cpu().numpy()

In [170]:
model = Net(len(db.columns)).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)#, momentum=0.5)

In [171]:
print("start ...")
print('*'*80)

'''
for each state with different nan number in row, e.g., 0, 1, 2, 4 ,5 
generate a trainable table(w/o nan by fill in with 0) and untrainable table(w/ nan even we fill in with 0) 
'''
max_nan_in_row =  db.isnull().sum(1).max()
print("max nan in row: ", max_nan_in_row)
for idx in range(max_nan_in_row):
    df = db.copy()
    
    print("\n\nstep 0: current state")
    print("deal with the original nan number(%d) in row"%idx)
    
    
    print("\n\nstep 1: find nan col and row in this state")
    col_had_nan = df.columns[df.isnull().any()].tolist()
#     for wnl in col_had_nan:
#         print(wnl)
    row_had_nan = df.isnull().any(axis=1)
    display(df.loc[row_had_nan,:].head())
    
    
    print("\n\nstep 2: extract non-nan row")
#     to_train = df.copy()
#     to_train.loc[row_had_nan, col_had_nan] = 0
#     display(to_train.loc[row_had_nan, :].head())
    to_train = df.loc[~row_had_nan, :]
    if to_train.shape[0] == 0:
        print("skip since no row")
        continue
    display(to_train.head())
    
    
    print("\n\nstep 3: normalization")
    train_mean = df[df.columns].mean()
    train_std  = df[df.columns].std()
    to_train.loc[:,to_train.columns] = (to_train[to_train.columns] - train_mean) / train_std
    to_train_numpy = to_train.to_numpy()
#     display(train_mean)
#     display(train_std)
    
#     to_gt_numpy = to_train_numpy.copy()
#     for i in range(to_train_numpy.shape[0]):
#         to_train_numpy[i].shape
#         indices = np.random.choice(np.arange(to_train_numpy[i].shape[0]), replace=False,
#                                size=int(to_train_numpy[i].shape[0] * random.uniform(0.1,0.5)))
#         to_train_numpy[i,indices]=0
    
    epochs = 5000
    train(model, to_train_numpy, idx+1, optimizer, epochs)
    display(to_train.head())
    
    
    print("\n\nstep 4: find nan row with %d nan in this state and test"%idx)
    row_had_1_nan = df.isnull().sum(axis=1) == idx+1
    to_test = df.loc[row_had_1_nan, :].copy()
    to_test.loc[:,to_test.columns] = (to_test[to_test.columns] - train_mean) / train_std
    mask = to_test.isnull()
    to_test.fillna(0, inplace=True)
    display(to_test.head())
    to_test_numpy = to_test.to_numpy()
    result = test(model, to_test_numpy)
#     pd.DataFrame(data=result, index=data[1:,0], columns=to_train.columns)
    display(df.loc[row_had_1_nan, :].head())
    result_db = to_test.copy()
    result_db[:] = result * (train_std.to_numpy()) + (train_mean.to_numpy())
#     print(result.shape)
#     print(train_std.to_numpy().shape)
#     print(train_mean.to_numpy().shape)
#     print(result * (train_std.to_numpy()) + (train_mean.to_numpy()))
    display(result_db.head())
    
    
    print("\n\nstep 5: replace the nan with generated values by our model")
#     db.loc[row_had_1_nan] = tt
#     display(db.loc[row_had_1_nan, :].head())
    db_tmp = db.loc[row_had_1_nan, :].copy()
    db_tmp[mask] = result_db[mask]
    db.loc[row_had_1_nan, :] = db_tmp
#     db.loc[row_had_1_nan, :][mask] = result_db[mask]
    display(db.loc[row_had_1_nan, :].head())
    


start ...
********************************************************************************
max nan in row:  4


step 0: current state
deal with the original nan number(0) in row


step 1: find nan col and row in this state


,Epoch (TDB),TP (TDB),e,i (deg),w (deg),Node (deg),q (AU),Q (AU),P (yr),MOID (AU),A1 (AU/d^2),A2 (AU/d^2),A3 (AU/d^2),DT (d)
0,54629,2455248.548,0.682527,4.894556,0.626838,295.985450,0.986192,5.23,5.48,0.027011,NaN,NaN,NaN,NaN
1,55101,2454741.329,0.666313,15.100746,203.649023,111.392003,1.190642,5.95,6.74,0.194101,NaN,NaN,NaN,NaN
2,49400,2446467.395,0.967143,162.262691,111.332485,58.420081,0.585978,35.08,75.32,0.063782,2.700000e-10,1.550000e-10,NaN,NaN
3,56870,2456618.204,0.848268,11.779995,186.540346,334.569806,0.336092,4.09,3.30,0.173092,1.580000e-10,-5.050000e-12,NaN,NaN
4,-9480,2390514.115,0.751299,13.216400,221.658800,250.669000,0.879073,6.19,6.65,0.000518,3.900000e-09,-2.540000e-10,NaN,NaN




step 2: extract non-nan row


,Epoch (TDB),TP (TDB),e,i (deg),w (deg),Node (deg),q (AU),Q (AU),P (yr),MOID (AU),A1 (AU/d^2),A2 (AU/d^2),A3 (AU/d^2),DT (d)
6,56981,2457053.028,0.637528,22.335015,172.506161,93.416327,1.239215,5.60,6.32,0.224191,7.860000e-11,-1.150000e-11,-1.530000e-10,131.00
13,56498,2455969.126,0.706818,31.908101,172.584425,195.397015,1.030696,6.00,6.59,0.035395,3.350000e-09,-4.290000e-10,-9.570000e-10,-32.80
20,56799,2456482.869,0.659203,11.757139,356.340205,82.164391,1.052262,5.12,5.43,0.068212,2.820000e-09,-1.360000e-09,6.650000e-10,-3.73




step 3: normalization


...

,Epoch (TDB),TP (TDB),e,i (deg),w (deg),Node (deg),q (AU),Q (AU),P (yr),MOID (AU),A1 (AU/d^2),A2 (AU/d^2),A3 (AU/d^2),DT (d)
0,0.253016,0.334173,-0.599288,-0.578655,-2.005476,1.733334,0.248193,-0.393510,-0.345908,-0.774206,0.0,0.0,0.0,0.0
1,0.310399,0.272627,-0.745717,-0.278369,0.307031,-0.160684,0.984454,-0.326949,-0.302520,0.620467,0.0,0.0,0.0,0.0
11,-1.496308,-1.496430,-0.960037,-0.200158,-0.121233,1.167880,1.278334,-0.336194,-0.303208,1.414494,0.0,0.0,0.0,0.0
12,-3.947235,-3.930974,1.543792,0.480406,-1.362440,2.267095,1.212665,1.899165,1.595905,2.909021,0.0,0.0,0.0,0.0
24,0.538958,0.528510,0.634587,-0.452884,1.835727,-0.933122,-0.479357,-0.147603,-0.223662,-0.288070,0.0,0.0,0.0,0.0


,Epoch (TDB),TP (TDB),e,i (deg),w (deg),Node (deg),q (AU),Q (AU),P (yr),MOID (AU),A1 (AU/d^2),A2 (AU/d^2),A3 (AU/d^2),DT (d)
0,54629,2455248.548,0.682527,4.894556,0.626838,295.985450,0.986192,5.23,5.48,0.027011,NaN,NaN,NaN,NaN
1,55101,2454741.329,0.666313,15.100746,203.649023,111.392003,1.190642,5.95,6.74,0.194101,NaN,NaN,NaN,NaN
11,40240,2440162.042,0.642581,17.758983,166.050417,240.875547,1.272248,5.85,6.72,0.289230,NaN,NaN,NaN,NaN
12,20080,2420098.290,0.919831,40.890061,57.080955,348.006447,1.254013,30.03,61.87,0.468283,NaN,NaN,NaN,NaN
24,56981,2456850.137,0.819155,9.169284,337.858042,36.109196,0.784161,7.89,9.03,0.085253,NaN,NaN,NaN,NaN


,Epoch (TDB),TP (TDB),e,i (deg),w (deg),Node (deg),q (AU),Q (AU),P (yr),MOID (AU),A1 (AU/d^2),A2 (AU/d^2),A3 (AU/d^2),DT (d)
0,55323.090673,2.455214e+06,0.661281,12.183886,75.371941,116.185160,1.062229,5.258532,5.792298,0.135971,1.085223e-08,9.798789e-10,2.498504e-10,89.585848
1,56827.613919,2.456993e+06,0.708023,9.863505,227.033900,227.285529,1.015544,5.724872,6.634441,0.154201,8.968508e-09,-6.860483e-10,7.107417e-10,40.098130
11,40093.802621,2.439671e+06,0.824444,19.463901,26.831175,94.394196,1.140087,12.865566,19.911337,0.340719,7.440409e-09,-1.985765e-09,2.269699e-10,96.240007
12,20376.941206,2.420852e+06,0.910343,44.065725,103.624335,134.276517,1.056752,28.989113,62.534125,0.403034,8.544481e-09,2.858628e-10,8.500603e-10,9.757171
24,53768.805801,2.453810e+06,0.693666,11.380781,149.201007,85.119715,0.936312,5.142209,5.386162,0.052341,1.628143e-08,1.996268e-09,2.682623e-10,10.252168




step 4: replace the nan with 888 since we will generate a value for it by our model


,Epoch (TDB),TP (TDB),e,i (deg),w (deg),Node (deg),q (AU),Q (AU),P (yr),MOID (AU),A1 (AU/d^2),A2 (AU/d^2),A3 (AU/d^2),DT (d)
0,54629,2455248.548,0.682527,4.894556,0.626838,295.985450,0.986192,5.23,5.48,0.027011,1.085223e-08,9.798789e-10,2.498504e-10,89.585848
1,55101,2454741.329,0.666313,15.100746,203.649023,111.392003,1.190642,5.95,6.74,0.194101,8.968508e-09,-6.860483e-10,7.107417e-10,40.098130
11,40240,2440162.042,0.642581,17.758983,166.050417,240.875547,1.272248,5.85,6.72,0.289230,7.440409e-09,-1.985765e-09,2.269699e-10,96.240007
12,20080,2420098.290,0.919831,40.890061,57.080955,348.006447,1.254013,30.03,61.87,0.468283,8.544481e-09,2.858628e-10,8.500603e-10,9.757171
24,56981,2456850.137,0.819155,9.169284,337.858042,36.109196,0.784161,7.89,9.03,0.085253,1.628143e-08,1.996268e-09,2.682623e-10,10.252168


In [152]:
torch.save(model.state_dict(),"the_most_best.pt")

In [151]:
display(db)

,Altitude (km),Velocity (km/s),Velocity Components (km/s): vx,Velocity Components (km/s): vy,Velocity Components (km/s): vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,38.000000,32.100000,3.000000,-17.000000,-27.000000,10000000000000,18.000
1,36.000000,17.552256,-10.700000,-7.600000,11.600000,139000000000,0.410
2,44.000000,20.007886,14.400000,4.600000,6.500000,309000000000,0.820
3,22.200000,16.200000,-2.300000,5.700000,16.500000,3820000000000,7.600
4,26.300000,13.585399,11.500000,-2.800000,-2.200000,33000000000,0.110
5,32.209458,17.631529,2.962279,-2.914471,-0.483275,73000000000,0.230
6,28.900000,17.053800,8.525665,-2.862404,-1.083376,34000000000,0.120
7,39.000000,17.549310,-28.200000,3.400000,4.600000,33000000000,0.110
8,39.000000,21.229346,2.589049,-8.504521,0.315017,72000000000,0.230
9,23.500000,13.271047,2.300000,2.500000,-11.300000,64000000000,0.200
